# Magnetic Force

M&I Ch 20 Magnetic Force

- §20.1 &ndash; Magnetic Force on a Moving Charge
- §20.3 &ndash; Combining Electric and Magnetic Forces
- §20.4 &ndash; The Hall Effect
- §20.5 &ndash; Motional EMF

In [ ]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [ ]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>
>   At a particular instant, a proton is traveling in the $+z$-direction, with 
>   speed $2 \times 10^{5}$ [m/s].
>   At the location of the proton, there is a magnetic field of magnitude $0.28$ [T] 
>   in the $+x$-direction, due to the current in a nearby coil.
>
>   __What is the direction of the magnetic force on the proton?__
>
>   (select one:)
>
>   - [ ] &nbsp; $+y$.
>
>   __What is the magnitude of the magnetic force on the proton?__
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert\vec{F}_{mag}\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{N}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]
>
>   At a particular instant, an electron is traveling in the $+z$-direction, 
>   with speed $9 \times 10^5$ [m/s]. 
>   At the location of the electron, there is a magnetic field of magnitude 
>   $0.25$ [T] in the $-x$ direction, due to a large bar magnet.
>
>   __What is the direction of the magnetic force on the electron?__
>
>   (select one:)
>
>   - [ ] &nbsp; $+y$.
>
>   __What is the magnitude of the magnetic force on the electron?__
>
>   &emsp;&emsp;&emsp;&emsp; $\lvert\vec{F}_{mag}\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{N}\right]$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   An electron and a proton are both in motion near each other, as shown in the
>   figure below. 
>
>   At the instant depicted, they are a distance $r$ apart.
>   The proton is moving with speed $v_p$, at an angle $\theta$ to the 
>   horizontal, and the electron is moving straight up with speed $v_e$.
>
>   (Assume the $+x$ direction is to the right, and the $+y$ direction is upward.
>   Indicate the direction with the sign of your answer.)
>
>   {{GRAPHIC}}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='3a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Calculate the $x$- and $y$- components of the force that the proton exerts on the electron.__
>
>   First, calculate the electric field and magnetic field that the proton produces at the location of the electron, then calculate the forces that these fields exert on the electron. 
>
>   Use the following as necessary for $F_{\textrm{net},\ x}$: $\epsilon_0$, $r$, $v_p$, $v_e$, $\theta$, and $c$. \
>   Use the following as necessary for $F_{\textrm{net},\ y}$: $\mu_0$, $e$, $r$, $v_p$, $v_e$, $\theta$, and $c$.
>
>   &emsp;&emsp;&emsp;&emsp; $F_{\textrm{net},\ x} = \fbox{\hspace{4em}?}$
>
>   &emsp;&emsp;&emsp;&emsp; $F_{\textrm{net},\ y} = \fbox{\hspace{4em}?}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Calculate the $x$- and $y$- components of the force that the electron exerts on the proton.__
>
>   First, calculate the electric field and magnetic field that the electron 
>   produces at the location of the proton, then calculate the forces that these
>   fields exert on the proton. (In a completely correct quantitative calculation, 
>   we would have to use the relativistically correct fields.)
>
>   Use the following as necessary for $F_{\textrm{net},\ x}$: $\epsilon_0$, $r$, $v_p$, $v_e$, $\theta$, and $c$. \
>   Use the following as necessary for $F_{\textrm{net},\ y}$: $\mu_0$, $e$, $r$, $v_p$, $v_e$, $\theta$, and $c$.
>
>   &emsp;&emsp;&emsp;&emsp; $F_{\textrm{net},\ x} = \fbox{\hspace{4em}?}$
>
>   &emsp;&emsp;&emsp;&emsp; $F_{\textrm{net},\ y} = \fbox{\hspace{4em}?}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(C)
>
>   Carefully consider your results. 
>
>   __Are the magnetic forces on the electron and proton equal and opposite?__
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>
>   __Does reciprocity hold for magnetic forces?__
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='3d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x00B3;&hairsp;&#x2E05;&hairsp;(D)
>
>   __Will the total momentum of the two particles remain constant?__
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>
>   __Is this a violation of conservation of momentum for an isolated system?__
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>
>   _Note:_ There is momentum associated with the electric and magnetic fields of these particles, as we will see in a later chapter.
>   Although the momentum of the proton plus the momentum of the electron does change, the sum of the particle moments plus the field momentum does _not_ change. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]*
>
>   __Mass Spectrometer__
>
>   A mass spectrometer is a tool used to accurately determine the mass of 
>   individual ionized atoms or molecules, or to separate atoms or molecules 
>   that have similar (but slightly different) masses. For example, you can 
>   deduce the age of a small sample of cloth from an ancient tomb, by using a 
>   mass spectrometer to determine the relative abundances of Carbon-14 (whose 
>   nucleus contains 6 protons and 8 neutrons), and Carbon-12 (the most common 
>   isotope of Carbon, whose nucleus contains 6 protons and 6 neutrons).
>
>   In organic material, the ratio of $^{14}\mathrm{C}$ to $^{12}\mathrm{C}$ 
>   depends on how old the material is, which is the basis for "Carbon-14 
>   Dating".  The isotope $^{14}\mathrm{C}$ is continually produced in the upper
>   atmosphere by nuclear reactions caused by "cosmic rays" (high-energy charged
>   particles from outer space, mainly protons), and $^{14}\mathrm{C}$ is 
>   radioactive with a half-life of 5700 years.
>
>   When a cotton plant is growing, some of the $\mathrm{CO}_2$ it extracts from
>   the air to build tissue contains $^{14}\mathrm{C}$, which has diffused down 
>   from the upper atmosphere.  However, after the cotton has been harvested, 
>   there is no further intake of $^{14}\mathrm{C}$ from the air, and the cosmic
>   rays that create $^{14}\mathrm{C}$ in the upper atmosphere can't penetrate 
>   the atmosphere and reach the cloth.  As a result, the amount of 
>   $^{14}\mathrm{C}$ in cotton cloth continually decreases with time, while the
>   amount of non-radioactive $^{12}\mathrm{C}$ remains constant.
>
>   <hr width="50%" style="border-top:1px dotted; border-bottom:none"/> 
>
>   A particular kind of mass spectrometer is shown in the figure below.  
>
>   {{ graphic }}
>
>   Carbon from the sample is ionized in the "ion source" at the left. The 
>   resulting singly ionized $^{12}\mathrm{C}^+$ and $^{14}\mathrm{C}^+$ ions 
>   have negligibly small initial velocities (and can be considered to be at 
>   rest).
>
>   Initially, the ions are accelerated through the potential difference 
>   $\Delta V_1$, then enter a region where the magnetic field has a fixed 
>   magnitude $\mathbf{B} = 0.16$ [T].  Within the region with non-zero magnetic
>   field, the ions pass through electric deflection plates that are 1 [cm] apart
>   and have a potential difference $\Delta V_2$; that is adjusted so that the 
>   electric deflection and the magnetic deflection cancel each other for a 
>   particular isotope -- One isotope goes straight through, while the other ion
>   is deflected and misses the entrance to the next section of the spectrometer.
>
>   The distance from the entrance to the fixed ion detector is a distance of 
>   $w = 27$ [cm].  There are controls that let you vary the __accelerating potential__ 
>   $\Delta V_1$, and the __deflecting potential__ $\Delta V_2$, so that only 
>   either $^{12}\mathrm{C}^+$ or $^{14}\mathrm{C}^+$ ions go all the way through
>   the system and reach the detector. The operator counts each kind of ion for 
>   fixed times, and thus determine the relative abundances.  The various 
>   deflections ensure that you count only the desired type of ion, for a 
>   particular setting of the two voltages. 
>   
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='4a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Which accelerating plate is positive?__
>
>   - [ ] &nbsp; left
>   - [ ] &nbsp; right
>
>   __Which deflecting plate is positive?__
>
>   - [ ] &nbsp; top
>   - [ ] &nbsp; bottom
>
>   __What is the direction of the magnetic field inside the spectrometer?__
>
>   - [ ] &nbsp; into the page
>   - [ ] &nbsp; out of the page
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(B)
>
>   __Determine the appropriate numerical values of $\Delta V_1$ and $\Delta V_2$ for $^{12}\mathrm{C}$.__
>
>   Carry out your intermediate calculations algebraically, so that you can use the algebraic results in the next part. 
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V_1 = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V_2 = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='4c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2074;&hairsp;&#x2E05;&hairsp;(C)
>
>   __Determine the appropriate numerical values of $\Delta V_1$ and $\Delta V_2$ for $^{14}\mathrm{C}$.__
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V_1 = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   &emsp;&emsp;&emsp;&emsp; $\Delta V_2 = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   An electron travels with velocity $\left\langle 2 \times 10^{5}, 0, 0 \right\rangle$ [m/s].  It enters a region in which there is a uniform magnetic field of $\mathrm{\vec{B}} = \left\langle 0, 0.9, 0 \right\rangle$ [T]. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='5a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the magnetic force on the electron?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{F}_\mathbf{B} = \left\langle\ \fbox{\hspace{8em}?}\ ,\ \fbox{\hspace{8em}?}\ ,\ \fbox{\hspace{8em}?}\ \right\rangle\ \left[\textrm{N}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(B)
>
>   Despite the magnetic force, the electron continues to travel in a straight 
>   line at constant speed. 
>   You conclude that there must be another force acting on the electron. 
>   Since you know there is also an electric field in this region, you decide 
>   that the other force must be an electric force. 
>
>   __What is the electric force?__
>
>   &emsp;&emsp;&emsp;&emsp; $\vec{F}_\mathbf{E} = \left\langle\ \fbox{\hspace{8em}?}\ ,\ \fbox{\hspace{8em}?}\ ,\ \fbox{\hspace{8em}?}\ \right\rangle\ \left[\textrm{N}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='5c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2075;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the electric field in this region, that is responsible for the electric force in part (B)?__
>
>   &emsp;&emsp;&emsp;&emsp; $\mathbf{\vec{E}} = \left\langle\ \fbox{\hspace{8em}?}\ ,\ \fbox{\hspace{8em}?}\ ,\ \fbox{\hspace{8em}?}\ \right\rangle\ \left[\textrm{N}/\mathrm{C}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]*
>
>   {{GRAPHIC}}
>
>   __Measuring properties of some material__
>
>   An experiment was carried out to determine the electrical properties of a 
>   new conducting material. 
>
>   A bar was made out of the material, $L = 15$ [cm] long with a rectangular 
>   cross-section $h = 5$ [cm] high and $w = 0.8$ [cm] deep.
>   The bar was part of a circuit, and carried a steady current (the diagram 
>   only shows part of the circuit). 
>   A uniform magnetic field of $1.7$ [T] was applied perpendicular to the bar, 
>   coming out of the page (using some coils that are not shown). 
>
>   Two voltmeters were connected, along and across the bar, as shown. 
>   The reading on Voltmeter 1 is $\Delta V_1 = +0.45$ volts, and the reading on
>   Voltmeter 2 is $\Delta V_2 = +0.00029$ volts. 
>   The connections across the bar for Voltmeter 2 were carefully placed directly 
>   across from each other, to eliminate false readings corresponding to the much 
>   larger voltage across the bar.
>   There is only one kind of mobile charge in this material. 
>
>   Remember that a voltmeter gives a positive reading, if the positive lead is 
>   connected to a higher potential location than the negative lead. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='6a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What is the sign of the mobile charges, and which way do they move?__
>
>   - [ ] &nbsp; Negative
>   - [ ] &nbsp; Positive
>
>   [...] mobile charges moving in the 
>
>   - [ ] &nbsp; $+x$
>   - [ ] &nbsp; $+y$
>   - [ ] &nbsp; $-x$
>   - [ ] &nbsp; $-y$
>
>   [...] direction. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(B)
>
>   __What is the drift speed $v$ of the mobile charges?__
>
>   &emsp;&emsp;&emsp;&emsp; $v = \fbox{\hspace{8em}?}\ \left[\textrm{m}/\textrm{s}\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the mobility $u$ of the mobile charges?__
>
>   &emsp;&emsp;&emsp;&emsp; $u = \fbox{\hspace{8em}?}\ \left[(\textrm{m}/\textrm{s}) / (\textrm{V}/\textrm{m})\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(D)
>
>   The current running through the bar was measured to be $0.7$ Amperes. 
>   If each mobile charge is singly charged $\left( \lvert q \rvert = e\right)$, 
>   __how many mobile charges are there in $1 \left[\textrm{m}^3\right]$ of this material?__
>
>   &emsp;&emsp;&emsp;&emsp; $N = \fbox{\hspace{8em}?}$ per cubic meter
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='6e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2076;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the resistance in ohms of this length of bar?__
>
>   &emsp;&emsp;&emsp;&emsp; $\fbox{\hspace{8em}?}\ \left[\Omega\right]$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='7' style="display:none">[7]</h2>

>   ---
>   ## [7]
>
>   A neutral iron bar is dragged to the left at speed $v$, through a region 
>   with a magnetic field $\mathbf{B}$ that points out of the page (see figure 
>   below).
>
>   {{GRAPHIC}}
>
>   __Which diagram (1-5) best shows the state of the bar?__
>
>   - [ ] &nbsp; 1
>   - [ ] &nbsp; 2
>   - [ ] &nbsp; 3
>   - [ ] &nbsp; 4
>   - [ ] &nbsp; 5
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   A neutral copper bar oriented horizontally moves downward through a region, 
>   where there is a magnetic field directed out of the page. 
>
>   {{GRAPHIC}}
>
>   __Which diagram correctly shows the distribution of charge on the bar?__
>
>   - [ ] &nbsp; 1
>   - [ ] &nbsp; 2
>   - [ ] &nbsp; 3
>   - [ ] &nbsp; 4
>   - [ ] &nbsp; 5
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]*
>
>   {{GRAPHIC}}
>
>   A neutral metal rod of length $0.25$ [m] slides horizontally at a constant 
>   speed of $8$ [m/s] on frictionless insulating rails, through a region of 
>   uniform magnetic field of magnitude $0.4$ [T] (directed out of the page, as 
>   shown in the diagram).
>
>   Before answering the following questions, draw a diagram showing the 
>   polarization of the rod, and the direction of the Coulomb electric field 
>   inside the rod. 
>
>   __<p align='right'>(10 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='9a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(A)
>
>   __Which of the following statements is true?__ 
>
>   (select one:)
>
>   - [ ] &nbsp; The right side of the moving rod is negative. 
>
>   - [ ] &nbsp; The top of the moving rod is negative.
>
>   - [ ] &nbsp; The right side of the moving rod is positive.
>
>   - [ ] &nbsp; The top of the moving rod is positive. 
>
>   - [ ] &nbsp; The moving rod is not polarized. 
>
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(B)
>
>   After the initial transit, __what is the magnitude of the net force on a mobile electron inside the rod?__
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \vec{F}_{\textrm{net}} \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{N}\right]$ 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9c' style="display:none">(C)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(C)
>
>   __What is the magnitude of the electric force on a mobile electron inside the rod?__
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \vec{F}_{\mathrm{E}} \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{N}\right]$ 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9d' style="display:none">(D)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(D)
>
>   __What is the magnitude of the magnetic force on a mobile electron inside the rod?__
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \vec{F}_{\mathrm{B}} \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{N}\right]$ 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9e' style="display:none">(E)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(E)
>
>   __What is the magnitude of the potential difference across the rod?__
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \Delta V \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$ 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='9f' style="display:none">(F)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>   
>   ### &#x2e04;&hairsp;&#x2079;&hairsp;&#x2E05;&hairsp;(F)
>
>   In what direction must you exert a force to keep the rod moving at constant speed? 
>
>   - [ ] &nbsp; To the left.
>   - [ ] &nbsp; No force is needed.
>   - [ ] &nbsp; To the right. 
>
>   &emsp;&emsp;&emsp;&emsp; $\left\lvert \Delta V \right\rvert = \fbox{\hspace{8em}?}\ \left[\textrm{V}\right]$ 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

---